# Tutorial 1: Gaze in Ego-Exo4D dataset

In this tutorial, we provide an introduction of the gaze data in Ego-Exo4D dataset, and a step-by-step guide on how to project 3D eye gaze to 2D in Egocentric/Exocentric view. 

Eye gaze is one of the 3D spatial signals provided by Ego-Exo4D dataset, which is pre-computed by Project Aria’s machine perception service (MPS). The gaze direction of the user is estimated as a single outward-facing ray anchored in-between the wearer’s eyes. Left and right eye gaze directions (yaw values) along with the depth at which these gaze directions intersect (translation values) are provided in the dataset. The convergence points and distances are derived from the predicted gaze directions. The combined direction’s yaw is used to populate the yaw field of the EyeGaze object for backwards compatibility. The pitch is common to left, right and combined gaze directions.

<img src="eye_gaze_032024_model.png" width="300"/>

Eye gaze data is located in each captures' or takes' eye_gaze folder. You can determine if a take or capture has eye gaze data by using `has_gaze` for a capture in <b>captures.json</b> and `has_trimmed_eye_gaze` for a take in <b>takes.json</b>. If eye gaze is available for the take, it is trimmed (cropped in time) with respect to that take. You will find three Eye Gaze MPS file outputs under the take folder: 
- `summary.json` - high level report on MPS eye gaze generation
- `general_eye_gaze.csv` - based on the standard eye gaze configuration
- `personalized_eye_gaze.csv` - only if the recording is made with <a href="https://facebookresearch.github.io/projectaria_tools/docs/ARK/mps/eye_gaze_calibration">in-session Eye Gaze Calibration</a>

### Prerequisites

#### Install libraries

Before we get started, we need to install necessary libraries. We will install the Python Package for Project Aria Tools. Please check <a href="https://facebookresearch.github.io/projectaria_tools/docs/data_utilities/installation/installation_python">Installation guide</a> for detailed instructions. We also need to install the rerun library for logging and data visualization.

In [ ]:
# install rerun
!pip install rerun-notebook==0.19.0
!pip install rerun-sdk==0.19.0
!pip install rerun-sdk[notebook]
# install project aria tools
!cd ~/projectaria_tools_python_env # Replace with your projectaria_tools_python_env folder
!python3 -m pip install --upgrade pip
!python3 -m pip install projectaria-tools'[all]'

We use Rerun to display temporal and interactive data. The following are some functions that we will use all across the tutorial.

In [13]:
import rerun as rr
import numpy as np

from projectaria_tools.core.calibration import CameraCalibration, DeviceCalibration
from projectaria_tools.core.sophus import SE3
from projectaria_tools.core import mps
from projectaria_tools.core.mps.utils import get_gaze_vector_reprojection
from projectaria_tools.core import data_provider
from projectaria_tools.utils.rerun_helpers import AriaGlassesOutline, ToTransform3D
from projectaria_tools.core import mps
from projectaria_tools.core.stream_id import StreamId
from projectaria_tools.core.sensor_data import TimeDomain, TimeQueryOptions
from projectaria_tools.core.calibration import CameraCalibration, KANNALA_BRANDT_K3
from tqdm import tqdm


def log_aria_glasses(
    device_calibration: DeviceCalibration,
    label: str,
    use_cad_calibration: bool = True
) -> None:
    ## Plot Project Aria Glasses outline (as lines)
    aria_glasses_point_outline = AriaGlassesOutline(
        device_calibration, use_cad_calibration
    )
    rr.log(label, rr.LineStrips3D([aria_glasses_point_outline]), timeless=True)

def log_calibration(
    camera_calibration: CameraCalibration,
    label: str
) -> None:
    rr.log(
        label,
        rr.Pinhole(
            resolution=[
                camera_calibration.get_image_size()[0],
                camera_calibration.get_image_size()[1],
            ],
            focal_length=float(
                camera_calibration.get_focal_lengths()[0]
            ),
        ),
        timeless=True,
    )

def log_pose(
    pose: SE3,
    label: str,
    timeless = False
) -> None:
    rr.log(
        label,
        ToTransform3D(pose, False),
        timeless = timeless
    )

def log_image(
    image_array : np.array,
    label: str,
    timeless = False
) -> None:
    rr.log(label, rr.DisconnectedSpace())
    rr.log(label, rr.Image(image_array), timeless = timeless)

def log_point_cloud(
    point_positions : np.array,
    label: str,
    timeless: bool = True) -> None:
    rr.log(label,rr.Points3D(point_positions, radii=0.001, colors=[200, 200, 200]), timeless=timeless)

#### Load one sample take and its corresponding 3D gaze 

In our tutorial, we will use the following take of bike repair as an example. You need to change `ego_exo_root` to the download directory for the Ego-Exo4D dataset.

In [2]:
ego_exo_root = '/datasets01/egoexo4d/v2/' # Replace with your cli's download directory for Ego-Exo4D
take_name = 'cmu_bike01_5'

import os
ego_exo_project_path = os.path.join(ego_exo_root, 'takes', take_name)
print(f'EgoExo Sequence: {ego_exo_project_path}')

if not os.path.exists(ego_exo_project_path):
    print("Please do update your path to a valid EgoExo sequence folder.")

EgoExo Sequence: /datasets01/egoexo4d/v2/takes/cmu_bike01_5


We retrieve the VRS data including device calibration collected by Aria glasses and plot sensors locations, orientations.
 - VRS: <a href="https://facebookresearch.github.io/vrs/">VRS</a> is the file format used to store the Project Aria Glasses multimodal data. VRS Data is stored Stream and are identified with a unique StreamId. `VrsDataProvider` enables you to list and retrieve all VRS data and calibration data.
 - DeviceCalibration: an interface that can be used to retrieve Intrinsics for Image Stream data (i.e Camera data) - `CameraCalibration` and Extrinsics are defined for all sensors - `SE3`.

Project Aria glasses use 3D Coordinate Frame Conventions. You can find an overview of these conventions <a href="3D Coordinate Frame Conventions">here</a> where Central Pupil Frame (CPF) and 3D Coordinate frame and system conventions are covered.

In [3]:
from projectaria_tools.core import data_provider
from projectaria_tools.utils.rerun_helpers import AriaGlassesOutline, ToTransform3D
RERUN_NOTEBOOK_ASSET="inline"
###
# We are using here the projectaria_tools API for:
# - retrieving the DeviceCalibration and the POSE of each sensor
# - we are then plotting those POSE onto the Aria glasses outline
###

##
# Retrieve device calibration and plot sensors locations, orientations
vrs_file_path = os.path.join(ego_exo_project_path, 'aria01.vrs')
print(f"VRS file path: {vrs_file_path}")
assert os.path.exists(vrs_file_path), "We are not finding the required vrs file"

vrs_data_provider = data_provider.create_vrs_data_provider(vrs_file_path)
if not vrs_data_provider:
    print("Couldn't create data vrs_data_provider from vrs file")
    exit(1)

device_calibration = vrs_data_provider.get_device_calibration()

log_aria_glasses(device_calibration, "world/device/glasses_outline")

# Plot CPF (Central Pupil Frame coordinate system)
T_device_CPF = device_calibration.get_transform_device_cpf()
log_pose(T_device_CPF, "device/CPF_CentralPupilFrame")

# Plot Project Aria Glasses outline (as lines)
log_aria_glasses(device_calibration, "device/glasses_outline")

VRS file path: /datasets01/egoexo4d/v2/takes/cmu_bike01_5/aria01.vrs


[ProgressLogger][INFO]: 2024-11-19 02:05:39: Opening /datasets01/egoexo4d/v2/takes/cmu_bike01_5/aria01.vrs...
[MultiRecordFileReader][DEBUG]: Opened file '/datasets01/egoexo4d/v2/takes/cmu_bike01_5/aria01.vrs' and assigned to reader #0
[VrsDataProvider][INFO]: streamId 211-1/camera-et activated
[VrsDataProvider][INFO]: streamId 214-1/camera-rgb activated
[VrsDataProvider][INFO]: streamId 231-1/mic activated
[VrsDataProvider][INFO]: streamId 247-1/baro0 activated
[VrsDataProvider][WARNING]: Unsupported TimeSync mode: APP, ignoring.
[VrsDataProvider][INFO]: Fail to activate streamId 286-1
[VrsDataProvider][INFO]: streamId 1201-1/camera-slam-left activated
[VrsDataProvider][INFO]: streamId 1201-2/camera-slam-right activated
[VrsDataProvider][INFO]: streamId 1202-1/imu-right activated
[VrsDataProvider][INFO]: streamId 1202-2/imu-left activated
[VrsDataProvider][INFO]: streamId 1203-1/mag0 activated


We can get RGB and SLAM left and right stream with a given StreamId.

In [4]:
rgb_stream_id = StreamId("214-1")
slam_left_stream_id = StreamId("1201-1")
slam_right_stream_id = StreamId("1201-2")
rgb_stream_label = vrs_data_provider.get_label_from_stream_id(rgb_stream_id)
slam_left_stream_label = vrs_data_provider.get_label_from_stream_id(slam_left_stream_id)
slam_right_stream_label = vrs_data_provider.get_label_from_stream_id(slam_right_stream_id)

We can further retrieve time domain, and image configurations from the stream.

In [5]:
# Init rerun api
rr.init("Aria Data Provider - Retrieve Image Stream data")
rec = rr.memory_recording()

# Configure option for data retrieval
time_domain = TimeDomain.DEVICE_TIME  # query data based on host time
option = TimeQueryOptions.CLOSEST # get data whose time [in TimeDomain] is CLOSEST to query time

# Retrieve Start and End time for the given Sensor Stream Id
start_time = vrs_data_provider.get_first_time_ns(rgb_stream_id, time_domain)
end_time = vrs_data_provider.get_last_time_ns(rgb_stream_id, time_domain)

# FYI, you can retrieve the Image configuration using the following
image_config = vrs_data_provider.get_image_configuration(rgb_stream_id)
width = image_config.image_width
height = image_config.image_height
print(f"StreamId {rgb_stream_id}, StreamLabel {rgb_stream_label}, ImageSize: {width, height}")

StreamId 214-1, StreamLabel camera-rgb, ImageSize: (1408, 1408)


Now let's visualize the stream from RGB, SLAM left and right using rerun. We sample 10 frames from the whole stream.

In [6]:
sample_count = 10
sample_timestamps = np.linspace(start_time, end_time, sample_count)
for sample in tqdm(sample_timestamps):

    # Retrieve the RGB image
    image_tuple_rgb = vrs_data_provider.get_image_data_by_time_ns(rgb_stream_id, int(sample), time_domain, option)
    timestamp = image_tuple_rgb[1].capture_timestamp_ns
    
    # Log timestamp as:
    # - device_time (so you can see the effective time between two frames)
    # - timestamp (so you can see the real VRS timestamp as INT value in the Rerun Timeline dropdown)
    rr.set_time_nanos("device_time", timestamp)
    rr.set_time_sequence("timestamp", timestamp)

    log_image(image_tuple_rgb[0].to_numpy_array(), f"vrs/{rgb_stream_label}")

    # Retrieving the SLAM images
    image_tuple_slam_left = vrs_data_provider.get_image_data_by_time_ns(slam_left_stream_id, int(sample), time_domain, option)
    log_image(image_tuple_slam_left[0].to_numpy_array(), f"vrs/{slam_left_stream_label}")

    image_tuple_slam_right = vrs_data_provider.get_image_data_by_time_ns(slam_right_stream_id, int(sample), time_domain, option)
    log_image(image_tuple_slam_right[0].to_numpy_array(), f"vrs/{slam_right_stream_label}")

# Showing the rerun window
rr.notebook_show()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 33.59it/s]


Viewer()

Ego Eye Gaze ray at a given timestamp T can be loaded using `mps_data_provider.get_personalized_eyegaze(timestamp)`.

In [7]:
# Init rerun api
rr.init("MPS - Trajectory data")
rec = rr.memory_recording()
rr.log("world", rr.ViewCoordinates.RIGHT_HAND_Z_UP, timeless=True)

## Configure the MpsDataProvider (interface used to retrieve Trajectory data)
mps_data_paths_provider = mps.MpsDataPathsProvider(ego_exo_project_path)
mps_data_paths = mps_data_paths_provider.get_data_paths()
mps_data_provider = mps.MpsDataProvider(mps_data_paths)

assert mps_data_provider.has_personalized_eyegaze(), "The sequence does not have Eye Gaze data"

# Log Glasses & calibration linked to the image we want to show
log_aria_glasses(device_calibration, "world/device/glasses_outline")
rgb_camera_calibration = device_calibration.get_camera_calib(rgb_stream_label)
slam_left_camera_calibration = device_calibration.get_camera_calib(slam_left_stream_label)
slam_right_camera_calibration = device_calibration.get_camera_calib(slam_right_stream_label)
log_calibration(rgb_camera_calibration, f"world/device/{rgb_stream_label}")
log_calibration(slam_left_camera_calibration, f"world/device/{slam_left_stream_label}")
log_calibration(slam_right_camera_calibration, f"world/device/{slam_right_stream_label}")

[MpsDataPathsProvider][WARNING]: Hand tracking folder (/datasets01/egoexo4d/v2/takes/cmu_bike01_5/hand_tracking) does not exist in MPS root folder, not loading wrist and palm poses.


In [8]:
###
# Eye Gaze data is independent of the Device pose
# 1. To display the eye gaze ray, you are applying the right relative transform Device_to_CPF
###
sample_count = 40
sample_timestamps = np.linspace(start_time, end_time, sample_count)
for sample in tqdm(sample_timestamps):
    image_tuple = vrs_data_provider.get_image_data_by_time_ns(rgb_stream_id, int(sample), time_domain, option)
    timestamp = image_tuple[1].capture_timestamp_ns
    rr.set_time_nanos("device_time", timestamp)
    rr.set_time_sequence("timestamp", timestamp)
    ##
    # Eye Gaze data
    # 1. Retrieve the eye_gaze data vector for a given timestamp
    # 2. Compute the corresponding 3D vector and retrieve its depth
    # 3. Reproject the eyegaze vector at Depth X on a given image (using Calibration data)
    ##

    # 1. Retrieve the eye_gaze data vector for a given timestamp
    eye_gaze = mps_data_provider.get_personalized_eyegaze(timestamp)

    # 2. Compute the corresponding 3D vector and retrieve its depth
    # Here is how to retrieve the depth of the EyeGaze vector
    # depth_m = eye_gaze.depth or 1.0
    # But here for display we are using a proxy of 30cm, so you can better see things in context of each other
    depth_m = 0.1
    gaze_vector_in_cpf = mps.get_eyegaze_point_at_depth(eye_gaze.yaw, eye_gaze.pitch, depth_m)
    gaze_vector_in_cpf = np.nan_to_num(gaze_vector_in_cpf)
    # Move EyeGaze vector to CPF coordinate system for visualization and log a 3D ray
    rr.log(
        "world/device/eye-gaze",
        rr.Arrows3D(
            origins=[T_device_CPF @ [0, 0, 0]],
            vectors=[T_device_CPF @ gaze_vector_in_cpf],
            colors=[[255, 0, 255]],
        ),
    )

    # 3. Reproject the eyegaze vector at Depth X on a given image (using Calibration data)
    # Compute eye_gaze vector at depth_m reprojection in the image
    depth_m = eye_gaze.depth or 1.0
    gaze_projection = get_gaze_vector_reprojection(
        eye_gaze,
        rgb_stream_label,
        device_calibration,
        rgb_camera_calibration,
        depth_m,
    )
    if gaze_projection is not None:
        rr.log(
            f"world/device/{rgb_stream_label}/eye-gaze_projection",
            rr.Points2D(gaze_projection, radii=30, colors=[0,255,0]),
        )
rr.notebook_show()

 35%|██████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                        | 14/40 [00:00<00:00, 67.09it/s]

Loaded #EyeGazes: 1555


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 72.28it/s]


Viewer()

### Projecting eye gaze from 3D to 2D in Egocentric View (CPF frame)

We project eye gaze data from egocentric view first. Eye gaze data is represented as a 3D ray with depth (showing the point of user focus). The eye gaze ray starts from the Central Pupil Frame(CPF). The reprojection of eye gaze ray in any Aria Image Stream (RGB, SLAMs) includes several steps:
 1. Use the `VrsDataProvider` to retrieve the RGB stream at a given timestamp
 2. Use the `MpsDataProvider` to retrieve if an EyeGaze file is available and to retrieve EyeGaze data at a given timestamp
 3. Compute the corresponding 3D eye gaze vector and retrieve its depth

In [33]:
rr.init("Eye Gaze - CPF - Image Reprojection")
rec = rr.memory_recording()
# Aria coordinate system sets X down, Z in front, Y Left
#rr.log("device", rr.ViewCoordinates.RIGHT_HAND_X_DOWN, timeless=True)

sample_count = 20
sample_timestamps = np.linspace(start_time, end_time, sample_count)
for sample in tqdm(sample_timestamps):

    # Retrieving the RGB image
    image_tuple_rgb = vrs_data_provider.get_image_data_by_time_ns(rgb_stream_id, int(sample), time_domain, option)
    timestamp = image_tuple_rgb[1].capture_timestamp_ns
    
    # Log timestamp as:
    # - device_time (so you can see the effective time between two frames)
    # - timestamp (so you can see the real VRS timestamp as INT value in the Rerun Timeline dropdown)
    rr.set_time_nanos("device_time", timestamp)
    rr.set_time_sequence("timestamp", timestamp)

    log_image(image_tuple_rgb[0].to_numpy_array(), f"device/{rgb_stream_label}")

    ##
    # Eye Gaze data
    # 1. Retrieve the eye_gaze data vector for a given timestamp
    # 2. Compute the corresponding 3D vector and retrieve its depth
    # 3. Reproject the eyegaze vector at Depth X on a given image (using Calibration data)
    ##

    # 1. Retrieve the eye_gaze data vector for a given timestamp
    eye_gaze = mps_data_provider.get_personalized_eyegaze(timestamp)

    # 2. Compute the corresponding 3D vector and retrieve its depth
    # Here is how to retrieve the depth of the EyeGaze vector
    # depth_m = eye_gaze.depth or 1.0
    # But here for display we are using a proxy of 30cm, so you can better see things in context of each other
    depth_m = 0.1
    gaze_vector_in_cpf = mps.get_eyegaze_point_at_depth(
        eye_gaze.yaw, eye_gaze.pitch, depth_m
    )
    gaze_vector_in_cpf = np.nan_to_num(gaze_vector_in_cpf)
    # Move EyeGaze vector to CPF coordinate system for visualization
    '''
    rr.log(
        "device/eye-gaze",
        rr.Arrows3D(
            origins=[T_device_CPF @ [0, 0, 0]],
            vectors=[T_device_CPF @ gaze_vector_in_cpf],
            colors=[[255, 0, 255]],
        ),
    )
    '''

    # 3. Reproject the eyegaze vector at Depth X on a given image (using Calibration data)
    # Compute eye_gaze vector at depth_m reprojection in the image
    depth_m = eye_gaze.depth or 1.0

    for stream_label in [rgb_stream_label]:
        if stream_label is rgb_stream_label:
            camera_calibration = rgb_camera_calibration
        else:
            camera_calibration = None

        gaze_projection = get_gaze_vector_reprojection(
            eye_gaze,
            stream_label,
            device_calibration,
            camera_calibration,
            depth_m,
        )
        if gaze_projection is not None:
            rr.log(
                f"device/{stream_label}/eye-gaze_projection",
                rr.Points2D(gaze_projection, radii=20),
            )


# Showing the rerun window
rr.notebook_show()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 62.14it/s]


Viewer()

### TODO: Projecting eye gaze from 3D to 2D in multiple Exocentric Views

In [31]:
# Create black images representing GoPro images

import math
import torchvision # to read video
from projectaria_tools.core.calibration import CameraCalibration, KANNALA_BRANDT_K3 # Aria/GoPro Camera Calibration
from projectaria_tools.core import mps

## Configure the MpsDataProvider (interface used to retrieve Trajectory data)
mps_data_paths_provider = mps.MpsDataPathsProvider(ego_exo_project_path)
mps_data_paths = mps_data_paths_provider.get_data_paths()
mps_data_provider = mps.MpsDataProvider(mps_data_paths)

# Init rerun api
rr.init("Ego_Exo - image reprojection")
rec = rr.memory_recording()

sample_count = 20  # sampling 20 frames from the videos

## Loading Exo Static camera calibration data
#
go_pro_proxy = []
static_calibrations = mps.read_static_camera_calibrations(os.path.join(ego_exo_project_path,"trajectory","gopro_calibs.csv"))
for static_calibration in static_calibrations:
    # assert the GoPro was correctly localized
    if static_calibration.quality != 1.0:
        print(f"Camera: {static_calibration.camera_uid} was not localized, ignoring this camera.")
        continue
    proxy = {}
    proxy["name"] = static_calibration.camera_uid
    proxy["image"] = zeros = np.zeros((static_calibration.height, static_calibration.width))
    proxy["images"] = []
    proxy["pose"] = static_calibration.transform_world_cam
    proxy["camera"] = CameraCalibration(
                            static_calibration.camera_uid,
                            KANNALA_BRANDT_K3,
                            static_calibration.intrinsics,
                            static_calibration.transform_world_cam,
                            static_calibration.width,
                            static_calibration.height,
                            None,
                            math.pi,
                            "")

    # Replace proxy image with an image from the gopro video
    video_path = os.path.join( ego_exo_project_path ,"frame_aligned_videos", static_calibration.camera_uid + ".mp4")
    reader = torchvision.io.VideoReader(video_path, "video")
    # Grab a frame at the middle of the video
    reader_metadata = reader.get_metadata()
    sample_timestamps = np.linspace(start_time, end_time, sample_count)
    for timestamp in sample_timestamps: 
        reader.seek(timestamp)
        frame = next(reader)['data'][0].numpy()
        proxy["images"].append(frame)
        
#        log_image(proxy["image"], f"image/{proxy['name']}")

    go_pro_proxy.append(proxy)


per_go_pro_reprojection = {}
# Sample the camera trajectory and reproject it on the GoPro images


sample_timestamps = np.linspace(start_time, end_time, sample_count)
for sample in tqdm(sample_timestamps):

    # Retrieving the RGB image
    image_tuple_rgb = vrs_data_provider.get_image_data_by_time_ns(rgb_stream_id, int(sample), time_domain, option)
    timestamp = image_tuple_rgb[1].capture_timestamp_ns
    
    # Log timestamp as:
    # - device_time (so you can see the effective time between two frames)
    # - timestamp (so you can see the real VRS timestamp as INT value in the Rerun Timeline dropdown)
    rr.set_time_nanos("device_time", timestamp)
    rr.set_time_sequence("timestamp", timestamp)

    for go_pro in go_pro_proxy:
        
        log_image(image, f"device/{go_pro['name']}")
        
    log_image(image_tuple_rgb[0].to_numpy_array(), f"device/{rgb_stream_label}")

    ##
    # Eye Gaze data
    # 1. Retrieve the eye_gaze data vector for a given timestamp
    # 2. Compute the corresponding 3D vector and retrieve its depth
    # 3. Reproject the eyegaze vector at Depth X on a given image (using Calibration data)
    ##

    # 1. Retrieve the eye_gaze data vector for a given timestamp
    eye_gaze = mps_data_provider.get_personalized_eyegaze(timestamp)

    # 2. Compute the corresponding 3D vector and retrieve its depth
    # Here is how to retrieve the depth of the EyeGaze vector
    # depth_m = eye_gaze.depth or 1.0
    # But here for display we are using a proxy of 30cm, so you can better see things in context of each other
    depth_m = 0.1
    gaze_vector_in_cpf = mps.get_eyegaze_point_at_depth(
        eye_gaze.yaw, eye_gaze.pitch, depth_m
    )
    gaze_vector_in_cpf = np.nan_to_num(gaze_vector_in_cpf)
    # Move EyeGaze vector to CPF coordinate system for visualization
    '''
    rr.log(
        "device/eye-gaze",
        rr.Arrows3D(
            origins=[T_device_CPF @ [0, 0, 0]],
            vectors=[T_device_CPF @ gaze_vector_in_cpf],
            colors=[[255, 0, 255]],
        ),
    )
    '''

    # 3. Reproject the eyegaze vector at Depth X on a given image (using Calibration data)
    # Compute eye_gaze vector at depth_m reprojection in the image
    depth_m = eye_gaze.depth or 1.0

    for stream_label in [rgb_stream_label]:
        if stream_label is rgb_stream_label:
            camera_calibration = rgb_camera_calibration
        else:
            camera_calibration = None

        gaze_projection = get_gaze_vector_reprojection(
            eye_gaze,
            stream_label,
            device_calibration,
            camera_calibration,
            depth_m,
        )
        if gaze_projection is not None:
            rr.log(
                f"device/{stream_label}/eye-gaze_projection",
                rr.Points2D(gaze_projection, radii=20),
            )


# Showing the rerun window
rr.notebook_show()

[MpsDataPathsProvider][WARNING]: Hand tracking folder (/datasets01/egoexo4d/v2/takes/cmu_bike01_5/hand_tracking) does not exist in MPS root folder, not loading wrist and palm poses.


Loaded #StaticCameraCalibration data: 4
TimeDomain.DEVICE_TIME


  5%|███████████▊                                                                                                                                                                                                                                | 1/20 [00:00<00:08,  2.12it/s]

Loaded #EyeGazes: 1555


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:09<00:00,  2.18it/s]


Viewer()

### Loading pre-computed 2D eye gaze on the egocentric frames